# 图像卷积
## 1 互相关运算
- 严格讲，卷积层所表达的运算实际为互相关运算
- 卷积层中，输入张量与核张量通过相关运算产生输出张量
- 实现`corr2d`：
  - 接收输入张量`X`、卷积核张量`K`
  - 输出张量`Y`

In [1]:
import torch
from torch import nn

def corr2d(X, K):
    """计算⼆维互相关运算"""
    h, w = K.shape
    Y = torch.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = (X[i:i + h, j:j + w] * K).sum()
    return Y

X = torch.tensor([[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]]) 
K = torch.tensor([[0.0, 1.0], [2.0, 3.0]]) 
corr2d(X, K)

tensor([[19., 25.],
        [37., 43.]])

## 2 卷积层
- 卷积层对输入和卷积核权重进行互相关运算，并在添加标量偏置后产生输出
- 卷积层中的两个被训练的参数是
  - 卷积核权重
  - 标量偏置

In [2]:
class Conv2D(nn.Module):
    def __init__(self, kernel_size):
        super().__init__() 
        self.weight = nn.Parameter(torch.rand(kernel_size)) 
        self.bias = nn.Parameter(torch.zeros(1))

def forward(self, x):
    return corr2d(x, self.weight) + self.bias

**图像中目标的边缘检测**
- 构造一个$6\times 8$像素的黑白图像：
  - 中间4列为黑色（0）
  - 其余为白色（1）
- 构造一个高度为1、宽度为2的卷积核`K`
- 相互运算时，若水平相邻的两元素相同，则输出为0，否则为非0
- 下例中，输出`Y`元素中：
  - 1: 白色到黑色的边缘
  - -1: 黑色到白色的边缘
  - 0: 其余

In [3]:
X = torch.ones((6, 8))
X[:, 2:6] = 0
print("X = \n", X)
K = torch.tensor([[1.0, -1.0]])
Y = corr2d(X, K)
print("Y = \n", Y)

X = 
 tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])
Y = 
 tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]])


**学习卷积核**
- 当有了更复杂数值的卷积核，或者 连续的卷积层时，我们不可能⼿动设计滤波器
- 下例探究可以通过仅查看“输⼊-输出”对来学习由`X`⽣成`Y`的卷积核
  - 先构造⼀个卷积层，并将其卷积核初始化为随机张量
  - 每次迭代中，⽐较`Y`与卷积层输出的平⽅误差，然后计算梯度来更新卷积核

In [4]:
# 构造⼀个⼆维卷积层，它具有1个输出通道和形状为（1，2）的卷积核 
conv2d = nn.Conv2d(1,1, kernel_size=(1, 2), bias=False)

# 这个⼆维卷积层使⽤四维输⼊和输出格式（批量⼤⼩、通道、⾼度、宽度）， 
# 其中批量⼤⼩和通道数都为1
X = X.reshape((1, 1, 6, 8)) 
Y = Y.reshape((1, 1, 6, 7)) 
lr = 3e-2 # 学习率

for i in range(20):
    Y_hat = conv2d(X)
    l = (Y_hat - Y) ** 2
    conv2d.zero_grad()
    l.sum().backward()
    # 迭代卷积核
    conv2d.weight.data[:] -= lr * conv2d.weight.grad
    if (i + 1) % 2 == 0:
        print(f'epoch {i+1}, loss {l.sum():.3f}')

conv2d.weight.data.reshape((1, 2))

epoch 2, loss 9.982
epoch 4, loss 2.849
epoch 6, loss 0.959
epoch 8, loss 0.358
epoch 10, loss 0.141
epoch 12, loss 0.057
epoch 14, loss 0.023
epoch 16, loss 0.009
epoch 18, loss 0.004
epoch 20, loss 0.002


tensor([[ 0.9958, -1.0040]])

# 多输⼊多输出通道

## 1 多输入通道

In [5]:
import torch

def corr2d_multi_in(X, K):
    # 先遍历“X”和“K”的第0个维度（通道维度），再把它们加在⼀起
    return sum(corr2d(x, k) for x, k in zip(X, K))

X = torch.tensor([[[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]], 
                  [[1.0, 2.0, 3.0], [4.0, 5.0, 6.0], [7.0, 8.0, 9.0]]])
K = torch.tensor([[[0.0, 1.0], [2.0, 3.0]], 
                  [[1.0, 2.0], [3.0, 4.0]]])

corr2d_multi_in(X, K)

tensor([[ 56.,  72.],
        [104., 120.]])

## 2 多输出通道

In [6]:
def corr2d_multi_in_out(X, K):
    # 迭代“K”的第0个维度，每次都对输⼊“X”执⾏互相关运算。 
    # 最后将所有结果都叠加在⼀起 
    return torch.stack([corr2d_multi_in(X, k) for k in K], 0)

K = torch.stack((K, K + 1, K + 2), 0) 
print("K.shape: ", K.shape)
corr2d_multi_in_out(X, K)

K.shape:  torch.Size([3, 2, 2, 2])


tensor([[[ 56.,  72.],
         [104., 120.]],

        [[ 76., 100.],
         [148., 172.]],

        [[ 96., 128.],
         [192., 224.]]])

# 汇聚层
## 最大/平均汇聚层

In [7]:
import torch 
from torch import nn

def pool2d(X, pool_size, mode='max'):
    p_h, p_w = pool_size 
    Y = torch.zeros((X.shape[0] - p_h + 1, X.shape[1] - p_w + 1)) 
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            if mode == 'max':
                Y[i, j] = X[i: i + p_h, j: j + p_w].max()
            elif mode == 'avg':
                Y[i, j] = X[i: i + p_h, j: j + p_w].mean() 
    return Y

X = torch.tensor([[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]])
print("X = \n", X)
print("max pooling: \n", pool2d(X, (2, 2)))
print("avg pooling: \n", pool2d(X, (2, 2), 'avg'))

X = 
 tensor([[0., 1., 2.],
        [3., 4., 5.],
        [6., 7., 8.]])
max pooling: 
 tensor([[4., 5.],
        [7., 8.]])
avg pooling: 
 tensor([[2., 3.],
        [5., 6.]])
